In [2]:
# default_exp core

# module name here

> API details.

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#exports

import numpy as np 
import pandas as pd
import pylab as pl
from ifis_tools import asynch_manager as am 
from ifis_tools import database_tools as db 
from ifis_tools import auxiliar as aux
from wmf import wmf


colors = ['#52270C','#A1623B','#E7A011','#F3DE0A']

def Get_sub_watersheds(watershed, order_max, order_min = 4):
    '''Obtains the sub-watersheds a different orders starting from the order_max and 
    ending on the order_min, there is no return, it only updates the watershed.Table'''
    orders = np.arange(order_max, order_min, -1).tolist()
    for Prun in orders:
        #Finds the connections points
        Ho = Prun
        watershed.Table['prun_'+str(Ho)] = 0
        idx = watershed.Table.loc[watershed.Table['h_order']>=Ho].index
        for i in idx:
            size = watershed.Table.loc[(watershed.Table['dest'] == i) & (watershed.Table['h_order'] >= Ho-1)]
            if size.shape[0] >= 2:
                watershed.Table.loc[size.index, 'prun_'+str(Ho)] = 1

        #Finds all the links that belong to a pruning level
        idx = watershed.Table.loc[watershed.Table['prun_'+str(Ho)] == 1].sort_values(by = ['Acum'], ascending = False).index
        cont = 2
        for i in idx:
            #Finds the watershed upstream
            t = am.hlmModel(linkid=i)
            idx_t = watershed.Table.index.intersection(t.Table.index)
            #Assign that pruning level to the sub-watershed
            watershed.Table.loc[idx_t, 'prun_'+str(Ho)] = cont
            #Go to next pruning level
            cont += 1
        print('Prun %d done' % Prun)
        
def cascade_values_async(Table, param, level, seed_values, norm_dev = 0.15):
    '''Generates a virtual setup of a watershed for a given set of values (seed_values)
    the result is updates on the cu.Table DataFrame. This is the asynchronic version
    that means that a level does not depends on the previous level.
    Params:
        - level: the minimum orton order to distribute the seed_values.
        - param: name of the parameter in the table.
        - seed_values: a list with the initial values for the biggest horton order.
        - norm_dev: deviation aprameter to produce the random cascade.
    Resutls:
        - cu.Table[param]'''
    Table[param] = 0.0
    
    for sub, value in zip(Table['prun_7'].unique(), seed_values):
        Table.at[Table['prun_7'] == sub, param] = value

    #Iterate for each level
    for level in np.arange(7,level,-1):
        name = 'prun_'+str(level)

        #Iterate through the sub-watersheds of each level `
        for subw in Table[name].unique():
            factor = np.random.normal(1, norm_dev)
            Table.loc[Table[name] == subw, param] *= factor  
            
def cascade_values_sync(Table, param, levels, seed_values, norm_dev = 0.15):
    '''Generates a virtual setup of a watershed for a given set of values (seed_values)
    the result is updates on the cu.Table DataFrame. Contrasted with the async version, this 
    version creates values that depends on the previous level.
    Params:
        - level: the minimum orton order to distribute the seed_values.
        - param: name of the parameter in the table.
        - seed_values: a list with the initial values for the biggest horton order.
        - norm_dev: deviation aprameter to produce the random cascade.
    Resutls:
        - cu.Table[param]'''
    p = param + '7'
    Table[p] = 0.0    
    for sub, value in zip(Table['prun_7'].unique(), seed_values):
        Table.at[Table['prun_7'] == sub, p] = value

    Factors = {}
    #Iterate for each level
    for level in np.arange(6,levels-1,-1):
        
        #Initial value
        name = 'prun_'+str(level)
        p = param + str(level)
        Table[p] = Table[param + str(level+1)].copy()
        fac = []
        
        #Iterate through the sub-watersheds of each level `
        for subw in Table[name].unique():
            factor = np.random.normal(1, norm_dev)            
            Table.loc[Table[name] == subw, p] *= factor            
            fac.append(factor)
        Factors.update({str(level): fac})
    return Factors

In [6]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_Virtual_data_setup.ipynb.
Converted index.ipynb.
